In [1]:
%%capture
import sys
import numpy as np

%pip install pyomo >/dev/null 2>/dev/null
%pip install highspy >/dev/null 2>/dev/null

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

In [2]:
print('Pyomo and Highs installed successfully.')

Pyomo and Highs installed successfully.
